# Read a SMILES and perform geometry optimization and frequency calculation at the GFN2-xTB level

This code is designed to automate the process of reading a SMILES string and its corresponding CID from an Excel file, generate a 3D structure and perform a preliminary geometry optimization using the Universal Force Field (UFF) within RDKit, perform a geometry optimization and frequency calculation using the GFN2-xTB semiempirical method using the `xtb` code from the Grimme group ([Link text](https://github.com/grimme-lab/xtb)), and organizing the outputs neatly into CID-specific folders. Using this code we were able to optimize 12796 molecules for a project in progress aimed to develop a geometry-aware graph neural network to predict toxicity.

In [ ]:
# I ran this locally on a desktop computer, RDKit and xtb should be previously installed.

# The following Python libraries and modules are required

import pandas as pd
import os
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.rdmolfiles import MolToXYZBlock

# Load Excel file containing SMILES and CIDs
df = pd.read_excel("smiles_excel.xlsx")

# Select a range of rows (e.g., from row 0 to 8 for 9 molecules)
df = df.iloc[:]  # Adjust the range as needed

# Base directory for output
base_output_dir = "xtb_ohess"
os.makedirs(base_output_dir, exist_ok=True)

# Function to process each SMILES string with its corresponding CID
def process_smiles(smiles, cid):
    try:
        # Create molecule directory using CID, formatted as an integer
        mol_dir = os.path.join(base_output_dir, f"CID_{int(cid)}")
        os.makedirs(mol_dir, exist_ok=True)

        # Define filenames within molecule directory
        xyz_filename = os.path.join(mol_dir, "structure.xyz")
        out_filename = os.path.join(mol_dir, "output.out")

        # Check if the XYZ file already exists to avoid overwriting
        if os.path.exists(xyz_filename):
            print(f"Skipping XYZ file creation for CID_{int(cid)} as it already exists.")
            return # Skip further processing for this molecule

        # Generate RDKit molecule object
        mol = Chem.MolFromSmiles(smiles)
        mol = Chem.AddHs(mol)  # Add explicit hydrogens

        # Generate 3D coordinates
        if AllChem.EmbedMolecule(mol, AllChem.ETKDG()) != 0:
            raise ValueError(f"RDKit failed to generate 3D coordinates for {smiles}")

        # Optimize geometry using UFF (Universal Force Field)
        AllChem.UFFOptimizeMolecule(mol)

        # Convert to XYZ format
        xyz_content = MolToXYZBlock(mol)

        # Save XYZ file
        with open(xyz_filename, "w") as f:
            f.write(xyz_content)

        print(f"XYZ file created: {xyz_filename}")

        # Run XTB optimization inside the molecule directory
        os.system(f"cd {mol_dir} && xtb structure.xyz --ohess > optimization.out")

    except Exception as e:
        print(f"Error processing SMILES {smiles} (CID: {cid}): {e}")

# Apply function to selected SMILES/CIDs (Excel file contains columns called "SMILES" and "CID")
for _, row in df.iterrows():
    smiles = row["SMILES"]
    cid = row["CID"]
    process_smiles(smiles, cid)

print("\nGeometry optimization of this batch is done.")